# i) Regra da soma

In [1]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [2]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [3]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, y_true = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [4]:
Pw = np.array([len(c)/2000 for c in partition])
Pw

array([0.1175, 0.116 , 0.1355, 0.1465, 0.0405, 0.01  , 0.1305, 0.151 ,
       0.0815, 0.071 ])

## Função de densidade

In [5]:
def calc_gaussian_density_prob(xk, d, means, var, cov_matrix):
    coef = np.power(2*np.pi, -d/2) 
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
    sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
    diff = xk - means
    exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
    exp_func = np.exp(exp_exp)
    
    return coef * sqrt_det_inv_cov * exp_func

In [32]:

def calc_gaussian_bayesian_data(x, partition):
    n,d = x.shape
    qtd_w = len(partition)
    means = np.array([x[idxs].mean(axis=0) for idxs in partition])
    var = np.array([((x[idxs]-means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
    cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

    for i in range(qtd_w):
        np.fill_diagonal(cov_matrix[i], var[i])
                
    p_x_w = np.empty((n, qtd_w))
    
    for k in range(n):
        for i in range(qtd_w):
            p_x_w[k, i] = calc_gaussian_density_prob(x[k], d, means[i], var[i], cov_matrix[i])   
    
    return p_x_w

def calc_prob_posteriori(p_x_w, Pw):
    qtd_x, qtd_w = p_x_w.shape 
    p_w_x = np.empty((qtd_x, qtd_w))
    
    for k in range(qtd_x):
        for i in range(qtd_w):
            sum_all = np.dot(p_x_w[k], Pw)
            p_w_x[k,i] = (p_x_w[k,i] * Pw[i])/sum_all
       
    return p_w_x


## Densidades por dataset

In [7]:
fac_byn_density_probs = calc_gaussian_bayesian_data(fac, partition)
fou_byn_density_probs = calc_gaussian_bayesian_data(fou, partition)
kar_byn_density_probs = calc_gaussian_bayesian_data(kar, partition)

In [8]:
fac_byn_density_probs[0,0], fou_byn_density_probs[0,0], kar_byn_density_probs[0,0]

(0.0, 3.243154665015897e+60, 7.254547591145725e-49)

## Prob. à priori por view

In [9]:
fac_byn_y_pred, fac_byn_posteriori_probs = calc_prob_posteriori(fac_byn_density_probs, Pw)
fou_byn_y_pred, fou_byn_posteriori_probs = calc_prob_posteriori(fou_byn_density_probs, Pw)
kar_byn_y_pred, kar_byn_posteriori_probs = calc_prob_posteriori(kar_byn_density_probs, Pw)


/home/thiago/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


## Regra da soma

Precisei tirar o fac porque seus valores são nulos. TENTAR CORRIGIR ISSO

In [20]:
def regra_soma_padrao(views, Pw):
    qtd_x = views[0].shape[0]
    qtd_w = len(Pw)
    x_sum_w = np.empty((qtd_x, qtd_w))

    for i in range(qtd_x):
        for k in range(qtd_w):
            views_sum = sum([v[i,k] for v in views])
            x_sum_w[i,k] = (1-len(views))*Pw[k] + views_sum
                    
    y_pred = x_sum_w.argmax(axis = 1) 
    return y_pred

## Estimador do scikit

In [11]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class ClassificaforBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
        self.partition = partition
        self.Pw = Pw
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self._fit_gaussian_bayesian_data(X)
        self.X_ = X
        self.y_ = y
        return self
    
    def _fit_gaussian_bayesian_data(self, X):
        n, d = X.shape
        qtd_w = len(self.partition)
        self.means = np.array([X[idxs].mean(axis=0) for idxs in self.partition])
        self.var = np.array([((X[idxs]-self.means[i])**2).mean(axis=0) for i, idxs in enumerate(self.partition)])
        self.cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

        for i in range(qtd_w):
            np.fill_diagonal(self.cov_matrix[i], self.var[i])

        return self
    
    def _calc_gaussian_density_prob(self, xk, cls):
        d = xk.shape[0]
        coef = np.power(2*np.pi, -d/2)
        inv_cov_matrix = np.linalg.inv(self.cov_matrix[cls])
        (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
        sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
        diff = xk - self.means[cls]
        exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
        exp_func = np.exp(exp_exp)

        return coef * sqrt_det_inv_cov * exp_func

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        
        desity_probs = np.empty((X.shape[0], len(self.classes_)))
        for k in range(desity_probs.shape[0]):
            for j in range(len(self.classes_)):
                desity_probs[k,j] = self._calc_gaussian_density_prob(X[k], j)
        
        post_probs = calc_prob_posteriori(desity_probs, self.Pw)
        
        return post_probs
        
        

In [12]:
class RegraSomaClasificadorBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
#         self.views = views
        self.partition = partition
        self.Pw = Pw
        self.clfs = []        
    
    def fit(self, X, y):
#         X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        
        for x in X:
            clf = ClassificaforBayesiano(self.partition, Pw)
            clf.fit(x,y)
            self.clfs.append(clf)
            
        return self
    
    def predict(self, X):
        assert len(X) == len(self.clfs)
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
#         X = check_array(X)
        
        post_probs = [clf.predict_proba(x) for clf, x in zip(self.clfs, X)]
        
        return self.regra_soma(post_probs, Pw=self.Pw)
    
    def get_params(self, deep=True):
        return {"Pw": self.Pw, "partition": self.partition}
    
    def regra_soma(self, matrizes, Pw):
        qtd_x = matrizes[0].shape[0]
        qtd_w = len(Pw)
        x_sum_w = np.empty((qtd_x, qtd_w))
        
        for i in range(qtd_x):
            for k in range(qtd_w):
                views_sum = sum([v[i,k] for v in matrizes])
                # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
                x_sum_w[i,k] = (1-len(matrizes))*Pw[k] + views_sum
                
        y_pred = x_sum_w.argmax(axis = 1) 
        return y_pred

# def regra_soma_padrao(fac_p_w_x, fou_p_w_x, kar_p_w_x, Pw):
# x_sum_w = np.empty((10, 2000))

# for i in range(10):
#     for k in range(2000):
#         views_sum =  fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         x_sum_w[i,k] = (1-2)*Pw[i] + views_sum

# y_pred = x_sum_w.argmax(axis = 0) 
# return y_pred

## Validação cruzada

In [13]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import RepeatedStratifiedKFold

RANDOM_SEED = 42
FOLDS = 10
REPEATS = 10

def get_splited_partition(idxs, y_true):
    partition = [[] for i in range(10)]
    
    for i,indice in enumerate(idxs):
        partition[y_true[indice]].append(i)
        
    return partition
    
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)
acc_scores = np.empty((cv.get_n_splits(),))
f1_scores = np.empty((cv.get_n_splits(),))

#views = [fac, fou, kar]
views = [fou, kar]

for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
    print(f"{i+1}/{cv.get_n_splits()}", end=" ")
    
    test_views = [v[test_index] for v in views]
    train_views = [v[train_index] for v in views]
    
    local_partition = get_splited_partition(train_index, y_true)
    clf = RegraSomaClasificadorBayesiano(local_partition, Pw)
    
    clf.fit(train_views, y_true[train_index])
    
    y_pred = clf.predict(test_views)
    
    acc_scores[i] = accuracy_score(y_true[test_index], y_pred)
    f1_scores[i] = f1_score(y_true[test_index], y_pred, average="macro") 
    
    if (i+1)%50 == 0:
        print
        print(f"\n\nAcurácia parcial: {acc_scores[:i+1].mean()} +/- ({acc_scores[:i+1].std()})")
        print(f"Medida-F parcial: {f1_scores[:i+1].mean()} +/- ({f1_scores[:i+1].std()})\n")


1/100 2/100 3/100 4/100 5/100 6/100 7/100 8/100 9/100 10/100 11/100 12/100 13/100 14/100 15/100 16/100 17/100 18/100 19/100 20/100 21/100 22/100 23/100 24/100 25/100 26/100 27/100 28/100 29/100 30/100 31/100 32/100 33/100 34/100 35/100 36/100 37/100 38/100 39/100 40/100 41/100 42/100 43/100 44/100 45/100 46/100 47/100 48/100 49/100 50/100 

Acurácia parcial: 0.7238 +/- (0.027010368379568616)
Medida-F parcial: 0.6575911422781199 +/- (0.031279227732342436)

51/100 52/100 53/100 54/100 55/100 56/100 57/100 58/100 59/100 60/100 61/100 62/100 63/100 64/100 65/100 66/100 67/100 68/100 69/100 70/100 71/100 72/100 73/100 74/100 75/100 76/100 77/100 78/100 79/100 80/100 81/100 82/100 83/100 84/100 85/100 86/100 87/100 88/100 89/100 90/100 91/100 92/100 93/100 94/100 95/100 96/100 97/100 98/100 99/100 100/100 

Acurácia parcial: 0.7235 +/- (0.026091186251299504)
Medida-F parcial: 0.6583459406076162 +/- (0.030174762922527264)



# K-Vizinhos

## Normalizando 


In [14]:
from sklearn.preprocessing import minmax_scale

fac_norm = minmax_scale(fac)
fou_norm = minmax_scale(fou)
kar_norm = minmax_scale(kar)

## Distâncias entre os elementos

In [15]:
from sklearn.metrics.pairwise import euclidean_distances

fac_dist = euclidean_distances(fac_norm, fac_norm)
fou_dist = euclidean_distances(fou_norm, fou_norm)
kar_dist = euclidean_distances(kar_norm, kar_norm)

## Função de densidade

In [30]:
def calc_knn_density_prob(view_dists, k, Pw, y_true):
    qtd_x = view_dists.shape[0]
    qtd_w = len(Pw)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    k_vizinhos = view_dists.argsort(axis=1)[:,:k+1]
    
    for j in range(qtd_x):
        w_vizinhos = y_true[k_vizinhos[j, 1:]]
        for i in range(qtd_w):
            p_x_w[j,i] = (w_vizinhos == i).sum()/k
                    
    return p_x_w
    

## Desidades por dataset

In [17]:
fac_knn_density_probs = calc_knn_density_prob(fac_dist, 5, Pw, y_true)
fou_knn_density_probs = calc_knn_density_prob(fou_dist, 5, Pw, y_true)
kar_knn_density_probs = calc_knn_density_prob(kar_dist, 5, Pw, y_true)


## Prob. à posteriori por view

In [18]:
_, fac_knn_posteriori_probs = calc_prob_posteriori(fac_knn_density_probs, Pw)
_, fou_knn_posteriori_probs = calc_prob_posteriori(fou_knn_density_probs, Pw)
_, kar_knn_posteriori_probs = calc_prob_posteriori(kar_knn_density_probs, Pw)

## Regra da soma

In [21]:
y_pred_knn_all = regra_soma_padrao([fac_knn_posteriori_probs, 
                                   fou_knn_posteriori_probs, 
                                   kar_knn_posteriori_probs], Pw)

print("Acurácia: ", accuracy_score(y_true, y_pred_knn_all))
print("Medida-F: ", f1_score(y_true, y_pred_knn_all, average="macro"))

Acurácia:  0.8125
Medida-F:  0.6759567552071272


## Validação cruzada

In [ ]:
cv = RepeatedStratifiedKFold(FOLDS, 10, RANDOM_SEED)
train_acc_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))

views = [fac_norm, fou_norm, kar_norm]
k = 5
for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
    print(f"{i+1}/{cv.get_n_splits()}", end=" ")
    
    test_views = [v[test_index] for v in views]
    train_views = [v[train_index] for v in views]
    train_views_dists = [euclidean_distances(v,v) for v in train_views]
    test_views_dists = [euclidean_distances(v,v) for v in test_views]
    
#     y_true_split = get_splited_partition(train_index, y_true)
    
    train_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[train_index]) 
                                 for v in train_views_dists]
    
    train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                              for v in train_views_density_probs]
    
    y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)
    
    train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)
    train_f1_scores[i] = f1_score(y_true[train_index], y_pred_train, average="macro") 
    
    if (i+1)%20 == 0:
        print(f"\n\nAcurácia parcial: {train_acc_scores[:i+1].mean()} +/- ({train_acc_scores[:i+1].std()})")
        print(f"Medida-F parcial: {train_f1_scores[:i+1].mean()} +/- ({train_f1_scores[:i+1].std()})\n")

    
    

1/100 2/100 3/100 4/100 5/100 6/100 7/100 8/100 9/100 10/100 11/100 12/100 13/100 14/100 15/100 16/100 17/100 18/100 19/100 20/100 

Acurácia parcial: 0.8083888888888889 +/- (0.003804237403504441)
Medida-F parcial: 0.6707240742627215 +/- (0.007462663578810034)

21/100 22/100 23/100 24/100 25/100 26/100 27/100 28/100 29/100 30/100 31/100 32/100 33/100 34/100 35/100 36/100 37/100 38/100 39/100 40/100 

Acurácia parcial: 0.8081805555555555 +/- (0.003922449434715664)
Medida-F parcial: 0.6705258842364112 +/- (0.006495137081714233)

41/100 42/100 43/100 44/100 45/100 46/100 47/100 48/100 49/100 50/100 51/100 52/100 53/100 54/100 55/100 56/100 57/100 58/100 59/100 60/100 

Acurácia parcial: 0.8084166666666667 +/- (0.004131113750920595)
Medida-F parcial: 0.6707166403459527 +/- (0.006392253215130073)

61/100 62/100 63/100 64/100 65/100 66/100 67/100 68/100 69/100 70/100 71/100 72/100 73/100 74/100 75/100 76/100 77/100 78/100 79/100 80/100 

Acurácia parcial: 0.8083750000000001 +/- (0.0044891630

# Janela de Parzen

## Função de densidade

In [ ]:
# def parzen_density_function(view, h, partition):
#     p_x_w = np.empty((10, 2000))
#     dims = view.shape[1]
#     elements_in_window = np.empty((10, dims))
    
#     def K(x):
#         return np.exp(-x**2/2)/np.sqrt(2*np.pi)

#     for i in range(10):
#         n = len(partition[i])
#         for k in range(2000):
#             p = np.array([np.array([K((view[k,j] - view[e,j])/h) for j in range(dims)]).prod() 
#                  for e in partition[i]]).sum()
            
#             p_x_w[i,k] = (1/(n*h**dims))
            
#     return p_x_w

def parzen_density_function(view, h, partition):
    p_x_w = np.empty((2000, 10))
    dims = view.shape[1]
    
    for i in range(10):
        n = len(partition[i])
        x_view = view[partition[i],:]
        
        for k in range(2000):
            diff = (view[k] - x_view)/h
            gaussian_diff = np.exp(-diff**2/2)/np.sqrt(2*np.pi)
            prod_dims = gaussian_diff.prod(axis=0)
            p_x_w[k,i] = (1/(n*h**dims)) * prod_dims.sum()
            print(p_x_w[k,i])
            
    return p_x_w

In [ ]:
fac_parzen_density_probs = parzen_density_function(fac, 50, partition)
fou_parzen_density_probs = parzen_density_function(fou, 50, partition)
kar_parzen_density_probs = parzen_density_function(kar, 50, partition)

## Probabilidade a priori

In [ ]:
fac_prazen_y_pred, fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
fou_prazen_y_pred, fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
fou_prazen_y_pred, kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

In [ ]:
y_prazen_pred_all = regra_soma_padrao(fac_prazen_posteriori_probs, 
                                    fou_prazen_posteriori_probs, 
                                    kar_prazen_posteriori_probs, Pw)

In [ ]:
set(y_prazen_pred_all)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

y_true = clustering.get_instances_class()

print("Acurácia: ", accuracy_score(y_true, y_prazen_pred_all))
print("Medida-F: ", f1_score(y_true, y_prazen_pred_all, average="macro"))